In [1]:
# NBA Stats scraping
#
# R. Andrew Fowler 
# Last update: march 14 2016

from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
import time
import re

####################################################

# SCRAPING FUNCTIONS

# Get the website text with a built in retry if 
# there is no connection.  Returns a string if it 
# tries more than a designated maximum number of 
# times.

def make_soup(url, maxTries=20):
    connection = 0
    tries = 0
    
    while not connection:
        
        try:
            html = urlopen(url).read()
            soup = BeautifulSoup(html,'html.parser')
            connection = 1
            tries += 1
            
        except:
            time.sleep(0.5)
            tries += 1
            
        if tries == maxTries:
            connection = 1
            soup = 'The soup is ruined!!'
    
    return soup

####################################################
#
# FUNCTION TESTING
#
# This area is a scratch pad for testing functions 
# after they have been written or changed.

g = 'http://m.bkref.com/m?p=XXteamsXX'
soupySoup = make_soup(g)

print(soupySoup.prettify())


<!DOCTYPE html>
<html class="mobile no-js">
 <head>
  <title>
   Team Index - m.bkref.com
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <link href="http://static.bkref.com" rel="dns-prefetch"/>
   <meta content="telephone=no" name="format-detection">
    <meta content="initial-scale=1.0,minimum-scale=1.0, maximum-scale=2.75" name="viewport">
     <link href="http://d2ft4b0ve1aur1.cloudfront.net/images-475/favicon-bbr.ico" rel="icon" type="image/vnd.microsoft.icon">
      <link href="http://d2ft4b0ve1aur1.cloudfront.net/images-475/favicon-bbr.ico" rel="SHORTCUT ICON" type="image/vnd.microsoft.icon">
       <link href="http://static.bkref.com/mobile/sr-bbr-mobile-min.220.css" rel="stylesheet"/>
       <link href="http://static.bkref.com/mobile/./icon-bbr.png" rel="apple-touch-icon"/>
       <link href="http://static.bkref.com/mobile/./startup-bbr.png" rel="apple-touch-startup-image"/>
       <script src="http://static.bkref.com/mobile/modernizr.js" t

In [2]:
# Make a list of the teams' names & abbreviations

# Using the mobile site because it's simpler
teamUrl = 'http://m.bkref.com/m?p=XXteamsXX'
teamSoup = make_soup(teamUrl)

# Everything is in one big table
teams = teamSoup.find('ul')

# Only active teams have an active link
activeTeams = teams.findAll('a')

teamNames = list()
teamAbrev = list()

for team in activeTeams:
    teamNames.append(team.getText())
    
    currLink = team['href']
    teamAbrev.append(currLink[13:16])

# Bundle all this up into a dataframe
teamFrame = pd.DataFrame({
        'Name':teamNames,
        'Abrv':teamAbrev})    
    
print(teamFrame)

   Abrv                    Name
0   ATL           Atlanta Hawks
1   BOS          Boston Celtics
2   NJN           Brooklyn Nets
3   CHA       Charlotte Hornets
4   CHI           Chicago Bulls
5   CLE     Cleveland Cavaliers
6   DAL        Dallas Mavericks
7   DEN          Denver Nuggets
8   DET         Detroit Pistons
9   GSW   Golden State Warriors
10  HOU         Houston Rockets
11  IND          Indiana Pacers
12  LAC    Los Angeles Clippers
13  LAL      Los Angeles Lakers
14  MEM       Memphis Grizzlies
15  MIA              Miami Heat
16  MIL         Milwaukee Bucks
17  MIN  Minnesota Timberwolves
18  NOH    New Orleans Pelicans
19  NYK         New York Knicks
20  OKC   Oklahoma City Thunder
21  ORL           Orlando Magic
22  PHI      Philadelphia 76ers
23  PHO            Phoenix Suns
24  POR  Portland Trail Blazers
25  SAC        Sacramento Kings
26  SAS       San Antonio Spurs
27  TOR         Toronto Raptors
28  UTA               Utah Jazz
29  WAS      Washington Wizards


In [2]:
# Get a link list for games:
cavsUrl ='http://www.basketball-reference.com/teams/CLE/2015_games.html'
cavSoup = make_soup(cavsUrl)

# Note that this will get all regular season and post season games
# for the cavs, this was 102 games in 2015
gameTable = cavSoup.findAll('tr')

linkList = list()

for game in gameTable:
    currGame = game.findAll('td')
    
    # Checks to make sure you're not on a divider line
    if len(currGame) > 1:
        # Boxscore link
        currLink = currGame[4].find('a')
        currLink = currLink['href']
        
        # Cuts up the link you scrape and redirects you to the play-by-play
        pbpLink = currLink[0:9] + '/pbp' + currLink[10:0]
        linkList.append(pbpLink)

print(linkList)

['/boxscore/pbp/201410300CLE.htm', '/boxscore/pbp/201410310CHI.htm', '/boxscore/pbp/201411040POR.htm', '/boxscore/pbp/201411050UTA.htm', '/boxscore/pbp/201411070DEN.htm', '/boxscore/pbp/201411100CLE.htm', '/boxscore/pbp/201411140BOS.htm', '/boxscore/pbp/201411150CLE.htm', '/boxscore/pbp/201411170CLE.htm', '/boxscore/pbp/201411190CLE.htm', '/boxscore/pbp/201411210WAS.htm', '/boxscore/pbp/201411220CLE.htm', '/boxscore/pbp/201411240CLE.htm', '/boxscore/pbp/201411260CLE.htm', '/boxscore/pbp/201411290CLE.htm', '/boxscore/pbp/201412020CLE.htm', '/boxscore/pbp/201412040NYK.htm', '/boxscore/pbp/201412050TOR.htm', '/boxscore/pbp/201412080BRK.htm', '/boxscore/pbp/201412090CLE.htm', '/boxscore/pbp/201412110OKC.htm', '/boxscore/pbp/201412120NOP.htm', '/boxscore/pbp/201412150CLE.htm', '/boxscore/pbp/201412170CLE.htm', '/boxscore/pbp/201412190CLE.htm', '/boxscore/pbp/201412210CLE.htm', '/boxscore/pbp/201412230CLE.htm', '/boxscore/pbp/201412250MIA.htm', '/boxscore/pbp/201412260ORL.htm', '/boxscore/pb

In [4]:
# Play by play analysis:
#
# Take a link from the link list and stick it 
# on the end of basketball-reference.com/

pbpUrl = 'http://www.basketball-reference.com/boxscores/pbp/201410300CLE.html'
pbpSoup = make_soup(pbpUrl).findAll('td',{'class':'align_right'})

# Regex string which finds game clock times
alwaysTime = re.compile('\d*:\d{2}\.\d')

maybeTime = list()
standardTime = list()

for thing in pbpSoup:
    maybeTime.append(thing.getText())
    
for times in maybeTime:
    if alwaysTime.search(times):
        standardTime.append(times)
    
print(standardTime)


['5:27.0', '21:52.0', '20:41.0', '2:53.0', '3:24.0', '12:00.0', '12:00.0', '11:47.0', '11:31.0', '11:20.0', '11:19.0', '11:16.0', '10:53.0', '10:34.0', '10:33.0', '10:21.0', '10:20.0', '10:03.0', '9:58.0', '9:40.0', '9:36.0', '9:17.0', '8:57.0', '8:41.0', '8:40.0', '8:31.0', '8:05.0', '8:00.0', '7:40.0', '7:31.0', '7:31.0', '7:18.0', '6:51.0', '6:50.0', '6:40.0', '6:35.0', '6:33.0', '6:30.0', '6:30.0', '6:30.0', '6:30.0', '6:29.0', '6:05.0', '6:04.0', '5:47.0', '5:45.0', '5:44.0', '5:31.0', '5:30.0', '5:23.0', '5:22.0', '5:15.0', '5:09.0', '4:45.0', '4:44.0', '4:37.0', '4:21.0', '3:53.0', '3:52.0', '3:45.0', '3:24.0', '3:20.0', '3:10.0', '3:09.0', '3:05.0', '3:04.0', '3:00.0', '2:49.0', '2:48.0', '2:43.0', '2:43.0', '2:43.0', '2:43.0', '2:43.0', '2:43.0', '2:43.0', '2:31.0', '2:30.0', '2:14.0', '1:57.0', '1:56.0', '1:51.0', '1:51.0', '1:40.0', '1:30.0', '1:30.0', '1:30.0', '1:25.0', '1:24.0', '1:17.0', '1:05.0', '1:05.0', '1:05.0', '1:05.0', '1:05.0', '1:05.0', '1:05.0', '0:49.0', '0:4

In [89]:
pbpUrl = 'http://www.basketball-reference.com/boxscores/pbp/201410300CLE.html'

pbpSoup = make_soup(pbpUrl).findAll('tr')

for rows in pbpSoup:
    currRow
print(pbpSoup)

[<tr class="valign_top">
<td>
<h1>New York Knicks at Cleveland Cavaliers Play-By-Play, October 30, 2014</h1>
<div class="clear_both width100 overflow_hidden"><div class="float_left"><table class="nav_table no_highlight stats_table">
<tr><th colspan="5">NBA Scores<br>Oct 30, 2014</br></th></tr>
<tr>
<td class="align_center background_yellow">
<table>
<tr>
<td><a href="/boxscores/201410300CLE.html">NYK<br>CLE</br></a></td>
<td class="align_right"><a href="/boxscores/201410300CLE.html">95<br>90</br></a></td>
</tr>
</table>
</td>
<td class="align_center">
<table>
<tr>
<td><a href="/boxscores/201410300DAL.html">UTA<br>DAL</br></a></td>
<td class="align_right"><a href="/boxscores/201410300DAL.html">102<br>120</br></a></td>
</tr>
</table>
</td>
<td class="align_center">
<table>
<tr>
<td><a href="/boxscores/201410300LAC.html">OKC<br>LAC</br></a></td>
<td class="align_right"><a href="/boxscores/201410300LAC.html">90<br>93</br></a></td>
</tr>
</table>
</td>
<td class="align_center">
<table>
<tr>

In [7]:
pbpUrl = 'http://www.basketball-reference.com/boxscores/pbp/201410300CLE.html'

pbpSoup = make_soup(pbpUrl)

pbpTable = pbpSoup.findAll('table',{'class':'no_highlight stats_table'})

print(pbpTable)


[<table class="no_highlight stats_table">
<tr id="q1">
<th colspan="6">1st Quarter</th></tr></table>]


In [2]:
fullDat = pd.DataFrame.from_csv('PBPComplete_20012015.csv')

In [3]:
print(fullDat.index)

DatetimeIndex(['2001-01-01', '2001-01-01', '2001-01-01', '2001-01-01',
               '2001-01-01', '2001-01-01', '2001-01-01', '2001-01-01',
               '2001-01-01', '2001-01-01',
               ...
               '2015-01-01', '2015-01-01', '2015-01-01', '2015-01-01',
               '2015-01-01', '2015-01-01', '2015-01-01', '2015-01-01',
               '2015-01-01', '2015-01-01'],
              dtype='datetime64[ns]', name='year', length=8466106, freq=None)


In [4]:
print(fullDat.columns)

Index(['game_id', 'away_team', 'home_team', 'quarter',
       'time_remaining_quarter', 'time_remaining_game',
       'time_remaining_in_seconds', 'score', 'away_score', 'home_score',
       'total_differential', 'score_change', 'timeout', 'timeout_called',
       'basket_made', 'basket_missed', 'rebound', 'assist', 'turnover', 'foul',
       'distance_from_basket', 'event'],
      dtype='object')


In [5]:
print(len(fullDat.index))

8466106


In [7]:
indexTup = tuple(fullDat.index)

print(len(indexTup))

8466106


In [ ]:
g = pd.DataFrame.